# PVT свойства углеводородных газов

<div style="text-align: right"> 
версия 0.1 <br>
Хабибуллин Ринат <br>
28.08.2017 
</div>


Описание ключевые свойств углеводородных газов необходимых для проведения расчетов

## Реальные газы
http://petrowiki.org/Real_gases

При относительно низких давлении и температуре объем занимаемым большинством реальных газов настолько велик, что что объем занимаемый молекулами газа пренебрежимо мал. Малы также и силы взаимодействия между молекулами. Поведение газов в таких условиях хорошо описывается соотношениями для [идельного газа](https://ru.wikipedia.org/wiki/%D0%98%D0%B4%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B3%D0%B0%D0%B7 "Википедия : Идеальный газ"). При повышении давления и температуры расстояние между молекулами уменьшается, доля объема занимаемого молекулами, а также силы взаимодействия между ними увеличиваются и должны учитываться при проведении расчетов. Такие газы, для которые предположения идеальности не выполняются, называют [реальными газами](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%B0%D0%BB%D1%8C%D0%BD%D1%8B%D0%B9_%D0%B3%D0%B0%D0%B7 "Википедия : Реальный газ").



### Уравнение состояния реального газа

$$
z=\frac{V_{real}(P,T)}{V_{ideal}(P,T)}
$$

Уравнение состояния для реального газа можно записать как 

$$
 pV=z \eta RT
$$

## Сжимаемость газа

In [8]:
import math
import numpy as np
from scipy.optimize import newton

In [9]:
def PseudoPressureStanding(gg):
    return 4.6 + 0.1 * gg - 0.258 * gg ** 2
    
def PseudoTemperatureStanding(gg):
    return 93.3 + 180.0 * gg - 6.94 * gg ** 2

def dranchuk_z_factor(T, p, T_cr, p_cr):
    """
    dranchuk's z-factor based on the newton's method
    """
    A = np.array([0.3265, -1.0700, -0.5339, 0.01569, -0.05165, 0.5475, -0.7361, 
                 0.1844, 0.1056, 0.6134, 0.7210])
    p_r=p/p_cr  #reduced pressure
    T_r=T/T_cr  #reduced temperature
    if not((0.0<=p_r<30.0 and 1.0<T_r<=3.0) or (p_r<1.0 and 0.7<T_r<=1.0)): #in the article 0.2<=p_r<30 is recommended
        print ('Warning! Temperature and pressure are out of range Dranchuck corr, p_r=', p_r, 'T_r=', T_r)
    C1=A[0]+A[1]/T_r+A[2]/T_r**3+A[3]/T_r**4+A[4]/T_r**5
    C2=A[5]+A[6]/T_r+A[7]/T_r**2
    C3=A[8]*(A[6]/T_r+A[7]/T_r**2)
    def dranchuk_eq(z):
        rho_r=0.27*p_r/(z*T_r) #reduced density
        return -z+1+C1*rho_r+C2*rho_r**2-C3*rho_r**5+A[9]*(1+A[10]*rho_r**2)*(rho_r**2/T_r**3)*np.exp(-A[10]*rho_r**2)
        
    def dranhuck_eq_deriv(z):
        D=0.27*p_r/z
        exp_cf=np.exp((-A[10]*D**2)/T_r**2)
        K=2*A[9]*A[10]*(1+A[10]*D**2/T_r**2)*D**4*exp_cf/(T_r**7)
        C1=D*(A[0]+A[1]/T_r+A[2]/T_r**3+A[3]/T_r**4+A[4]/T_r**5)/(z*T_r)
        C2=(2*D**2)*(A[5]+A[6]/T_r+A[7]/T_r**2)/(z*T_r**2)
        C3=(5*A[8]*D**5)*(A[6]/T_r+A[7]/T_r**2)/(z*T_r**5)
        C4=A[9]*D**2*exp_cf*(2+4*A[10]*D**2/T_r**2)/(z*T_r**5)
        return -1-C1-C2+C3-C4+K/z
        
    solution=newton(dranchuk_eq, 1, dranhuck_eq_deriv)
    return solution

In [10]:
dranchuk_z_factor(2,1,1,1)

0.96738929184997624